# Imports
All problem-speciffic files (code) in in the folder `MeshpLaplace2D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [1]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from pLaplace2D.jax_energy import J
from pLaplace2D.mesh import MeshpLaplace2D
from IPython.display import display, HTML
import pandas as pd

from jax import config

config.update("jax_enable_x64", True)

# Setup of the problem

In [2]:
results_df = pd.DataFrame(columns=['Time', 'Nit', 'Fun'])
times_asembly = []
times_solver = []

for mesh_level in range(2, 10):

    mesh = MeshpLaplace2D(mesh_level=mesh_level)
    params, adjacency, u_init = mesh.get_data_jax()
    energy_pLaplace = EnergyDerivator(J, params, adjacency, u_init)

    F, dF, ddF = energy_pLaplace.get_derivatives()
    ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg" if mesh_level > 6 else "direct", verbose=False, tol=1e-3)
    res = newton(F, dF, ddf_with_solver, u_init, verbose=False, tolf=1e-5, linesearch_tol=1e-3)
    # Create a DataFrame for the current result, round the results, and concatenate it with the main DataFrame
    time_asembly = energy_pLaplace.timings['compilation'] + energy_pLaplace.timings['coloring']
    times_asembly.append(time_asembly)
    times_solver.append(res['time'])

    current_result_df = pd.DataFrame([{
        'Time': f"{res['time']:.2f} / {time_asembly:.2f}",
        'Nit': res['nit'],
        'Fun': round(float(res['fun']), 4)
    }])
    results_df = pd.concat([results_df, current_result_df], ignore_index=True)

# Display the DataFrame as HTML in Jupyter Notebook
results_df.index = range(2,10)  # Reset index to simple counter
display(HTML(results_df.to_html()))

# Apply rounding for LaTeX export using Styler.format
latex_style = results_df.style.format({
    'Fun': "{:.4f}"
})

print(latex_style.to_latex())

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


,Time,Nit,Fun
2,0.01 / 0.18,4,-7.3411
3,0.01 / 0.17,4,-7.7767
4,0.02 / 0.17,5,-7.9051
5,0.07 / 0.25,6,-7.9430
6,0.30 / 0.23,6,-7.9546
7,0.86 / 0.42,6,-7.9583
8,4.14 / 1.20,8,-7.9596
9,19.96 / 4.41,9,-7.9600


\begin{tabular}{lllr}
 & Time & Nit & Fun \\
2 & 0.01 / 0.18 & 4 & -7.3411 \\
3 & 0.01 / 0.17 & 4 & -7.7767 \\
4 & 0.02 / 0.17 & 5 & -7.9051 \\
5 & 0.07 / 0.25 & 6 & -7.9430 \\
6 & 0.30 / 0.23 & 6 & -7.9546 \\
7 & 0.86 / 0.42 & 6 & -7.9583 \\
8 & 4.14 / 1.20 & 8 & -7.9596 \\
9 & 19.96 / 4.41 & 9 & -7.9600 \\
\end{tabular}

